# HAR (human activity recognition) using DNN/DL on fNIRS data

In [1]:
% ls /home/arasdar/datasets/fNIRs-data-10subjects/

P11-4-17-2018/  P13-4-17-2018/  P15-4-18-2018/  P17-4-18-2018/  P19-4-19-2018/
P12-4-17-2018/  P14-4-18-2018/  P16-4-18-2018/  P18-4-19-2018/  P20-4-19-2018/


In [2]:
% ls /home/arasdar/datasets/fNIRs-data-10subjects/P12-4-17-2018/

1. Right Hand/  2. Both Hands/  3. Left Hand/  4. Right Leg/  5. Left Leg/


In [3]:
% ls /home/arasdar/datasets/fNIRs-data-10subjects/P12-4-17-2018/1.\ Right\ Hand

2018-04-17_006/
fNIR_data.txt
head20180417-145130.txt
NIRS-2018-04-17_006_deoxyhb_T141to2511_C1to20.txt
NIRS-2018-04-17_006_oxyhb_T141to2511_C1to20.txt
Processed/
r_hand20180417-145128.txt
r_lower_arm20180417-145129.txt
r_upper_arm20180417-145129.txt


In [4]:
import numpy as np
import os
import pandas as pd

# % find ../../datasets/fNIRs_data/ | grep fNIR_data # NOT WORKING!!
def find_all(name, path):
    result = []
    for root, dirs, files in os.walk(path):
        if name in files:
            result.append(os.path.join(root, name))
    return result

allpaths = find_all(name='fNIR_data.txt', path='/home/arasdar/datasets/fNIRs-data-10subjects/')
allpaths = sorted(allpaths, reverse=False)
# print(allpaths, len(allpaths))
# allpaths, len(allpaths)

In [5]:
# df: data frame object
df = []
for each_idx in range(len(allpaths)):
    file = pd.read_csv(filepath_or_buffer=allpaths[each_idx], names=['time', 'sample', 
                       'channel', 'channel', 'channel', 'channel', 'channel',
                       'channel', 'channel', 'channel', 'channel', 'channel',
                       'channel', 'channel', 'channel', 'channel', 'channel',
                       'channel', 'channel', 'channel', 'channel', 'channel',
                       'channel', 'channel', 'channel', 'channel', 'channel',
                       'channel', 'channel', 'channel', 'channel', 'channel',
                       'channel', 'channel', 'channel', 'channel', 'channel',
                       'channel', 'channel', 'channel', 'channel', 'channel'],
                         header=None)
    df.append(file)
    
for each in range(len(df)):
#     print(df[each].shape, allpaths[each])
    df[each]=df[each].drop(axis=1, columns=None, index=None, labels=['time', 'sample'])
    df[each] = df[each].dropna()
    df[each]['channel.39'] = df[each]['channel.39'].astype(str).str[1:-1].astype(float)
# print(len(df))

/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/pandas/io/parsers.py:678: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)


In [6]:
data, labels = [], []
for each in range(0, len(df), 1):
    dfmat = df[each].as_matrix()
    label = allpaths[each][59:60]
#     print(dfmat.dtype, dfmat.shape, label, allpaths[each])
    data.append(dfmat)
    labels.append(label)
len(data), len(labels)

/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


(48, 48)

In [7]:
# This is very much like a convolution for extracting the windows
# size/width, stride/overlap, padding, dilation, num filters/out channel
def minibatching(X, Y, stride, width):
    Xmb, Ymb = [], []
    print(len(X), len(Y))
    # 1st and 1st
    for eachX in range(len(X)):
        num_mb = ((X[eachX].shape[0]-width)//stride)+1
        for each in range(num_mb):
            # The max is (num_mb-1)*stride+width==X[idx].shape[0]
            # The last each is (num_mb-1)
            # each = ((each-1)*stride)+width
            each *= stride
            Xmb.append(X[eachX][each:each+width])
            # There is only one label for one image signal or signal window or temporal window
            #Ymb.append(Y[eachX][each:each+1])
            Ymb.append(Y[eachX])
    return Xmb, Ymb

In [8]:
# Width is based on the sampling rate which is roughly about 233 points per window
# for 10sec rest and 20 sec activity
width = 250
Xmb, Ymb = minibatching(X=data, Y=labels, stride=1, width=width)
# for eachX, eachY in zip(Xmb, Ymb):
#     print(eachX.shape, eachY)
print(len(Xmb), len(Ymb))
print(Xmb[0].shape, Xmb[0].dtype)
print(Ymb[0])

48 48
69615 69615
(250, 40) float64
1


In [9]:
# Conversion from python list to numpy array
X, Y=np.array(object=Xmb, dtype=float), np.array(object=Ymb, dtype=int)
print(X.shape, X.dtype, Y.shape, Y.dtype)

(69615, 250, 40) float64 (69615,) int64


In [10]:
# Now I should devide the data into train and test
# Train and valid split
from sklearn.model_selection import train_test_split

# 30% of the training data/ entire training data is assigned to validation.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.30)
print(Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape)
print(Xtrain.dtype, Xtest.dtype, Ytrain.dtype, Ytest.dtype)

(48730, 250, 40) (20885, 250, 40) (48730,) (20885,)
float64 float64 int64 int64


In [11]:
# # # standardizing/normalizing the train and test data
# # def standardize(train, test):
# # """ Standardize data """
# # # Standardize train and test
# # X_train = (train - np.mean(train, axis=0)[None,:,:]) / np.std(train, axis=0)[None,:,:]
# # X_test = (test - np.mean(test, axis=0)[None,:,:]) / np.std(test, axis=0)[None,:,:]
# # return X_train, X_test

# Xtrain = (Xtrain - Xtrain.mean(axis=0))/ Xtrain.std(axis=0)
# Xtest = (Xtest - Xtest.mean(axis=0))/ Xtest.std(axis=0)
# print(Xtrain.shape, Xtrain.dtype)
# print(Xtest.shape, Xtest.dtype)

In [12]:
print(Xtrain.mean(axis=0), Xtrain.std(axis=0))

[[0.23554403 0.20575593 0.18495657 ... 0.46111169 0.52762557 0.49465244]
 [0.23556481 0.20576707 0.184952   ... 0.46112577 0.52764148 0.49466631]
 [0.23560505 0.20574727 0.18494023 ... 0.46114088 0.52765266 0.49468089]
 ...
 [0.23552294 0.20630135 0.18543658 ... 0.46198475 0.52880979 0.49748372]
 [0.23552992 0.20628603 0.18538877 ... 0.46200798 0.52884831 0.49750334]
 [0.23552453 0.2063183  0.1854071  ... 0.46201101 0.52883827 0.49752433]] [[0.19285766 0.16627797 0.22238958 ... 0.42500892 0.32865339 0.39384347]
 [0.19285166 0.16628782 0.22234756 ... 0.42500321 0.32866523 0.3938663 ]
 [0.19287132 0.16628347 0.22240013 ... 0.42503077 0.32869325 0.39389741]
 ...
 [0.19349291 0.16647666 0.22302495 ... 0.42435421 0.33020757 0.39728052]
 [0.19350172 0.16645575 0.22301906 ... 0.42437603 0.33022773 0.39730335]
 [0.19351622 0.16648958 0.22311172 ... 0.42442223 0.33022996 0.39732096]]


In [13]:
print(Xtest.mean(axis=0), Xtest.std(axis=0))

[[0.23765635 0.20597665 0.18336603 ... 0.45654725 0.52858353 0.49686509]
 [0.23760658 0.20595644 0.18338596 ... 0.45652995 0.52855008 0.49686592]
 [0.23750989 0.20600986 0.18342205 ... 0.45651175 0.52852138 0.49686913]
 ...
 [0.237542   0.20650866 0.18383809 ... 0.45759895 0.52984235 0.49954631]
 [0.23752772 0.20654956 0.1839596  ... 0.45754887 0.52976889 0.49950552]
 [0.23754784 0.20648215 0.18392217 ... 0.45755178 0.52980125 0.49946098]] [[0.19329449 0.16668512 0.21998131 ... 0.42141658 0.3307093  0.39560504]
 [0.19331477 0.16665267 0.22009097 ... 0.4214354  0.33068433 0.39561556]
 [0.1932717  0.16665732 0.21997587 ... 0.42136722 0.33061238 0.39560948]
 ...
 [0.19404805 0.16667976 0.22077867 ... 0.42120564 0.33241725 0.39893713]
 [0.19402672 0.16673311 0.2207919  ... 0.4211549  0.33240221 0.39889508]
 [0.19400099 0.16666572 0.2205828  ... 0.42105012 0.33242536 0.3988653 ]]


In [14]:
# Now separating train and validation set
# 30% of the training data/ entire training data is assigned to validation.
Xtrain, Xvalid, Ytrain, Yvalid = train_test_split(Xtrain, Ytrain, test_size=0.30)
print(Xtrain.shape, Xvalid.shape, Xtest.shape, Xtrain.dtype, Xvalid.dtype, Xtest.dtype)
print(Ytrain.shape, Yvalid.shape, Ytest.shape, Ytrain.dtype, Yvalid.dtype, Ytest.dtype)

(34111, 250, 40) (14619, 250, 40) (20885, 250, 40) float64 float64 float64
(34111,) (14619,) (20885,) int64 int64 int64


In [15]:
# In this one we should define and detect GPUs for tensorflow
# GPUs or CPU
import tensorflow as tf

# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.8.0
Default GPU Device: /device:GPU:0


In [16]:
def model_input(input_size, output_size):
    #     N, W, Cin = Xvalid.shape[0], Xvalid.shape[1], Xvalid.shape[2]
    Xinputs = tf.placeholder(dtype=tf.float32, shape=[None, *input_size], name='Xinputs')
    
    #     N, Cout = Yvalid.shape[0], Yvalid.shape[1]
    Yindices = tf.placeholder(dtype=tf.int32, shape=[None], name='Yindices')
    
    # # Batchnorm mode: training and inference/testing/validation
    # #is_bn_training = tf.placeholder(dtype=tf.bool, shape=[], name='is_bn_training')
    # training = tf.placeholder(dtype=tf.bool, shape=[], name='training')

    # returning input data/sequences, output labels/classes
    return Xinputs, Yindices

In [17]:
# tf.layers.conv2d(
#     inputs,
#     filters,
#     kernel_size, kHxkW
#     strides=(1, 1), sHxsW
#     padding='valid',
#     data_format='channels_last', NxHxWxC
#     data_format='channels_first', NxCxHxW
#     dilation_rate=(1, 1),
#     activation=None,
#     use_bias=True,
#     kernel_initializer=None,
#     bias_initializer=tf.zeros_initializer(),
#     kernel_regularizer=None,
#     bias_regularizer=None,
#     activity_regularizer=None,
#     kernel_constraint=None,
#     bias_constraint=None,
#     trainable=True,
#     name=None,
#     reuse=None
# )

In [ ]:
tf.layers.conv2d(
    inputs,
    filters,
    kernel_size=(0, 3), #kHxkW= [] list/tuple; can be a single integer too 3 which means 3x3
    strides=(0, 2), #sHxsW; default=(1, 1); for reducing dim and getting rid of pooling
    padding='valid', # or 'same'
    data_format='channels_last', #NxHxWxC default
#     data_format='channels_first', #NxCxHxW
    dilation_rate=(0, 1), #dHxdW= (1, 1) default
)

In [18]:
# Discriminator/ classifier/ recognizer
def discriminator(Xinputs, input_size, output_size, hidden_size, reuse=False, alpha=0.1, training=True):
    with tf.variable_scope('discriminator', reuse=reuse):
        
        # Flatten/Vectorize the input data tensor for FC/fully connected layer/Dense Layer
        Xinputs_vec = tf.reshape(tensor=Xinputs, shape=[-1, input_size[0]*input_size[1]])
        
        # First fully connected layer
        h1 = tf.layers.dense(inputs=Xinputs_vec, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=output_size)   
        #predictions = tf.nn.softmax(logits)
        
        # return output logits for loss and accuracy
        return logits

In [19]:
# # Discriminator/ classifier/ recognizer
# def discriminator(Xinputs, input_size, output_size, hidden_size, reuse=False, alpha=0.1, training=True):
#     with tf.variable_scope('discriminator', reuse=reuse):
        
#         # Flatten/Vectorize the input data tensor for FC/fully connected layer/Dense Layer
#         Xinputs_vec = tf.reshape(tensor=Xinputs, shape=[-1, input_size[0]*input_size[1]])
        
#         # First fully connected layer
#         h1 = tf.layers.dense(inputs=Xinputs_vec, units=hidden_size)
#         nl1 = tf.maximum(alpha * h1, h1)
        
#         # Second fully connected layer
#         h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
#         nl2 = tf.maximum(alpha * h2, h2)
        
#         # Output layer
#         logits = tf.layers.dense(inputs=nl2, units=output_size)   
#         #predictions = tf.nn.softmax(logits)
        
#         # return output logits for loss and accuracy
#         return logits

In [20]:
# This is the forward propagation of the model to calculate the loss.
def model_loss(Xinputs, Yindices, input_size, output_size, hidden_size):
    
    # Creating logits and labels
    Ylogits = discriminator(Xinputs=Xinputs, input_size=input_size, output_size=output_size, 
                            hidden_size=hidden_size)
    Ylabels = tf.one_hot(indices=Yindices, depth=output_size, dtype=Ylogits.dtype)
    
    # Loss using logits and labels
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=Ylogits, labels=Ylabels))
    
    # Accuracy using logits and labels
    acc_tensor = tf.equal(x=tf.argmax(axis=1, input=Ylogits), y=tf.argmax(axis=1, input=Ylabels))
    acc = tf.reduce_mean(axis=0, input_tensor=tf.cast(dtype=tf.float32, x=acc_tensor))

    # returning loss and accuracy
    return loss, acc

In [21]:
# def model_opt(loss, learning_rate):
#     """
#     Get optimization operations in order
#     :param loss: Discriminator/classifier loss Tensor
#     :param learning_rate: Learning Rate Placeholder
#     :return: A tuple of (discriminator training)
#     """
#     # Get weights and bias to update
#     t_vars = tf.trainable_variables()
#     # q_vars = [var for var in t_vars if var.name.startswith('qfunction')] # Q: action At/at
#     # g_vars = [var for var in t_vars if var.name.startswith('generator')] # G: next state St/st
#     # d_vars = [var for var in t_vars if var.name.startswith('discriminator')] # D: reward Rt/rt
#     var_list = [var for var in t_vars if var.name.startswith('discriminator')] # D: reward Rt/rt
    
#     # Optimize
#     with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
#         # q_opt = tf.train.AdamOptimizer(learning_rate).minimize(q_loss, var_list=q_vars)
#         # g_opt = tf.train.AdamOptimizer(learning_rate).minimize(g_loss, var_list=g_vars)
#         # d_opt = tf.train.AdamOptimizer(learning_rate).minimize(d_loss, var_list=d_vars)
#         opt = tf.train.AdamOptimizer(learning_rate).minimize(loss, var_list=var_list)

#     return opt

In [22]:
class MLP:
    def __init__(self, input_size, output_size, hidden_size, learning_rate):

        # Data of the Model: make the data available inside the framework
        self.Xinputs, self.Yindices = model_input(input_size=input_size, output_size=output_size)

        # Create the Model: calculating the loss and forwad pass
        self.loss, self.acc = model_loss(Xinputs=self.Xinputs, Yindices=self.Yindices,
                                         input_size=input_size, output_size=output_size, hidden_size=hidden_size)

        # Update the model: backward pass and backprop
        #self.opt = model_opt(loss=self.loss, learning_rate=learning_rate)
        self.opt = tf.train.AdamOptimizer(learning_rate).minimize(self.loss)

In [23]:
def get_batches(X, Y, batch_size):
    """ Return a generator for batches """
    n_batches = len(X) // batch_size
    X, Y = X[:n_batches*batch_size], Y[:n_batches*batch_size]

    # Loop over batches and yield
    for b in range(0, len(X), batch_size):
        yield X[b:b+batch_size], Y[b:b+batch_size]

In [24]:
# Network Hyper-parameters

# # Input
# Xwidth, Xchannels = Xvalid.shape[1], Xvalid.shape[2]

# Output layer
assert Ytrain.max()==Ytest.max()==Yvalid.max(), 'Output classes'
# Ychannels = Yvalid.max()

# Hidden layer
input_size = [Xvalid.shape[1], Xvalid.shape[2]]
hidden_size = Xvalid.shape[1]* Xvalid.shape[2]
output_size = Yvalid.max()

# learning parameters
batch_size = Xvalid.shape[0]//1 # experience mini-batch size
train_epochs = 1000              # max number of training episodes/epochs
learning_rate = 0.001            # learning rate for training/optimization/adam

In [25]:
print('Yvalid.max(), Yvalid.min():', Yvalid.max(), Yvalid.min())
print('Yvalid.shape:', Yvalid.shape)
print('Xvalid.shape:', Xvalid.shape)

Yvalid.max(), Yvalid.min(): 5 1
Yvalid.shape: (14619,)
Xvalid.shape: (14619, 250, 40)


In [26]:
tf.reset_default_graph()

model = MLP(input_size=input_size, hidden_size=hidden_size, output_size=output_size, learning_rate=learning_rate)

In [ ]:
# We should save the after training and validation
saver = tf.train.Saver() 

# Loss and accuracy of the model for training and validation
train_loss_mean, valid_loss_mean = [], []
train_acc_mean, valid_acc_mean = [], []

# now that we can calculate loss and optimize, we can start a session for calculating the error.
with tf.Session() as sess:
    
    # Initialize all the model parameters/variables
    sess.run(fetches=tf.global_variables_initializer())
    
    #     # Restoring/loading/uploading the trained and validated model
    #     saver.restore(sess,'checkpoints/mlp-fnirs-har.ckpt')
    
    # for every epoch start feeding the arrays into the tensors in the model
    for epoch in range(train_epochs):
        train_loss, valid_loss = [], []
        train_acc, valid_acc = [], []
        
        # Training
        for Xinputs, Yindices in get_batches(X=Xtrain, Y=Ytrain, batch_size=batch_size):
            feed_dict = {model.Xinputs: Xinputs, model.Yindices: Yindices}
            loss, acc, _ = sess.run(fetches=[model.loss, model.acc, model.opt], feed_dict=feed_dict)
            train_loss.append(loss)
            train_acc.append(acc)
            
        # printing out train and validation loss
        print('epoch:', epoch+1, 'train_loss:', np.mean(train_loss), 'train_acc:', np.mean(train_acc))
        
        # Saving the losses for plotting
        train_loss_mean.append(np.mean(train_loss))
        train_acc_mean.append(np.mean(train_acc))
        
        # Validation
        for Xinputs, Yindices in get_batches(X=Xvalid, Y=Yvalid, batch_size=batch_size):
            feed_dict = {model.Xinputs: Xinputs, model.Yindices: Yindices}
            loss, acc = sess.run(fetches=[model.loss, model.acc], feed_dict=feed_dict)
            valid_loss.append(loss)
            valid_acc.append(acc)
        
        # printing out train and validation loss
        print('epoch:', epoch+1, 'valid_loss:', np.mean(valid_loss), 'valid_acc:', np.mean(valid_acc))

        # Saving the losses for plotting
        valid_loss_mean.append(np.mean(valid_loss))
        valid_acc_mean.append(np.mean(valid_acc))
        
        #         # printing out train and validation loss
        #         print('epoch:', epoch+1, 
        #               'train_loss:', np.mean(train_loss), 'valid_loss:', np.mean(valid_loss),
        #               'train_acc:', np.mean(train_acc), 'valid_acc:', np.mean(valid_acc))
        
    
    # Saving the trained and validated model
    saver.save(sess,'checkpoints/mlp-fnirs-har.ckpt')

epoch: 1 train_loss: 6.7945423 train_acc: 0.20760652
epoch: 1 valid_loss: 13.018539 valid_acc: 0.23531021
epoch: 2 train_loss: 11.511367 train_acc: 0.27060676
epoch: 2 valid_loss: 7.2873964 valid_acc: 0.31821603
epoch: 3 train_loss: 6.666421 train_acc: 0.3046378
epoch: 3 valid_loss: 6.0155897 valid_acc: 0.34167865
epoch: 4 train_loss: 5.5748844 train_acc: 0.32813463
epoch: 4 valid_loss: 4.47367 valid_acc: 0.41965935
epoch: 5 train_loss: 4.048591 train_acc: 0.39106643
epoch: 5 valid_loss: 3.5599895 valid_acc: 0.3676038
epoch: 6 train_loss: 3.143035 train_acc: 0.4101854
epoch: 6 valid_loss: 2.8266127 valid_acc: 0.3825843
epoch: 7 train_loss: 2.523176 train_acc: 0.36681718
epoch: 7 valid_loss: 1.9985411 valid_acc: 0.40652576
epoch: 8 train_loss: 1.8668896 train_acc: 0.3531705
epoch: 8 valid_loss: 1.8317511 valid_acc: 0.36527807
epoch: 9 train_loss: 1.7085202 train_acc: 0.3826185
epoch: 9 valid_loss: 1.4748303 valid_acc: 0.417402
epoch: 10 train_loss: 1.3289514 train_acc: 0.46118066
epoch:

epoch: 77 valid_loss: 0.75442064 valid_acc: 0.69286543
epoch: 78 train_loss: 0.9204589 train_acc: 0.6736781
epoch: 78 valid_loss: 0.86517453 valid_acc: 0.62877077
epoch: 79 train_loss: 0.7828111 train_acc: 0.635748
epoch: 79 valid_loss: 1.2191778 valid_acc: 0.6061974
epoch: 80 train_loss: 0.8372953 train_acc: 0.65137833
epoch: 80 valid_loss: 0.5044117 valid_acc: 0.68575144
epoch: 81 train_loss: 0.5100134 train_acc: 0.6741569
epoch: 81 valid_loss: 0.7100329 valid_acc: 0.6439565
epoch: 82 train_loss: 0.8458938 train_acc: 0.6257952
epoch: 82 valid_loss: 0.9507564 valid_acc: 0.6851358
epoch: 83 train_loss: 0.81695604 train_acc: 0.6787058
epoch: 83 valid_loss: 0.872762 valid_acc: 0.62644506
epoch: 84 train_loss: 0.7108462 train_acc: 0.64518774
epoch: 84 valid_loss: 0.90447736 valid_acc: 0.632533
epoch: 85 train_loss: 0.7749181 train_acc: 0.66040766
epoch: 85 valid_loss: 1.056708 valid_acc: 0.628634
epoch: 86 train_loss: 0.9066402 train_acc: 0.64857376
epoch: 86 valid_loss: 0.6635743 valid_a

In [ ]:
import matplotlib.pyplot as mplot
%matplotlib inline

mplot.plot(train_loss_mean, label='train_loss_mean')
mplot.plot(valid_loss_mean, label='valid_loss_mean')
mplot.legend()

In [ ]:
mplot.plot(train_acc_mean, label='train_acc_mean')
mplot.plot(valid_acc_mean, label='valid_acc_mean')
mplot.legend()

In [ ]:
with tf.Session() as sess:
    sess.run(fetches=tf.global_variables_initializer())
    
    # Restoring/loading/uploading the trained and validated model
    saver.restore(sess,'checkpoints/mlp-fnirs-har.ckpt')
    
    # Saving the test loss for every batch/minibtch
    test_loss, test_acc = [], []
    
    # Testing
    for Xinputs, Yindices in get_batches(X=Xtest, Y=Ytest, batch_size=batch_size):
        feed_dict = {model.Xinputs: Xinputs, model.Yindices: Yindices}
        loss, acc = sess.run(fetches=[model.loss, model.acc], feed_dict=feed_dict)
        test_loss.append(loss)
        test_acc.append(acc)
        
    # Printing the test loss
    print('test_loss:', np.mean(test_loss), 'test acc', np.mean(test_acc))

In [ ]:
# Latest test_loss: 0.7203058 test acc 0.67275465